In [2]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin

s = requests.Session()
s.headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36"

def get_all_forms(url):
    """Given a `url`, it returns all forms from the HTML content"""
    soup = bs(s.get(url).content, "html.parser")
    return soup.find_all("form")
details = {}

def get_form_details(form):
    """
    This function extracts all possible useful information about an HTML `form`
    """
    global details
    # get the form action (target url)
    try:
        action = form.attrs.get("action", "").lower()
    except:
        action = None
    # get the form method (POST, GET, etc.)
    method = form.attrs.get("method", "get").lower()
    # get all the input details such as type and name
    inputs = []
    for input_tag in form.find_all("input"):
        input_type = input_tag.attrs.get("type", "text")
        input_name = input_tag.attrs.get("name")
        inputs.append({"type": input_type, "name": input_name})
    # put everything to the resulting dictionary
    details["action"] = action
    details["method"] = method
    details["inputs"] = inputs
    return details

In [3]:
print(get_all_forms("https://knaw.nl"))

[<form action="/zoeken" method="get" role="search"><input autofocus="" name="query" placeholder="Begin met zoeken..." type="text"/><div class="sc-48e0fc29-0 cZwhZB" size="32"><svg fill="none" height="32" viewbox="0 0 32 32" width="32" xmlns="http://www.w3.org/2000/svg"> <path clip-rule="evenodd" d="M5 13C5 8.58172 8.58172 5 13 5C17.4183 5 21 8.58172 21 13C21 17.4183 17.4183 21 13 21C8.58172 21 5 17.4183 5 13ZM13 3C7.47715 3 3 7.47715 3 13C3 18.5228 7.47715 23 13 23C15.4013 23 17.6049 22.1536 19.3287 20.7429L26.293 27.7072C26.6835 28.0977 27.3167 28.0977 27.7072 27.7072C28.0977 27.3167 28.0977 26.6835 27.7072 26.293L20.7429 19.3287C22.1536 17.6049 23 15.4013 23 13C23 7.47715 18.5228 3 13 3Z" fill-rule="evenodd"></path> </svg> </div><button class="sc-f789eb3-0 fSPVMq search__submit" type="submit"><span class="btn__text">Zoeken</span></button></form>]


In [11]:
def is_vulnerable(response):
    """A simple boolean function that determines whether a page 
    is SQL Injection vulnerable from its `response`"""
    errors = {
        # MySQL
        "you have an error in your sql syntax;",
        "warning: mysql",
        # SQL Server
        "unclosed quotation mark after the character string",
        # Oracle
        "quoted string not properly terminated",
    }
    for error in errors:
        # if you find one of these errors, return True
        if error in response.content.decode().lower():
            return True
    # no error detected
    return False

def scan_sql_injection(url):
    # test on URL
    for c in "\"'":
        # add quote/double quote character to the URL
        new_url = f"{url}{c}"
        print("[!] Trying", new_url)
        # make the HTTP request
        res = s.get(new_url)
        if is_vulnerable(res):
            # SQL Injection detected on the URL itself, 
            # no need to preceed for extracting forms and submitting them
            print("[+] SQL Injection vulnerability detected, link:", new_url)
            return
    # test on HTML forms
    forms = get_all_forms(url)
    print(f"[+] Detected {len(forms)} forms on {url}.")
    for form in forms:
        form_details = get_form_details(form)
        for c in "\"'":
            # the data body we want to submit
            data = {}
            for input_tag in form_details["inputs"]:
                if input_tag["type"] == "hidden" or input_tag["value"]:
                    # any input form that is hidden or has some value,
                    # just use it in the form body
                    try:
                        data[input_tag["name"]] = input_tag["value"] + c
                    except:
                        pass
                elif input_tag["type"] != "submit":
                    # all others except submit, use some junk data with special character
                    data[input_tag["name"]] = f"test{c}"
            # join the url with the action (form request URL)
            url = urljoin(url, form_details["action"])
            if form_details["method"] == "post":
                res = s.post(url, data=data)
            elif form_details["method"] == "get":
                res = s.get(url, params=data)
            # test whether the resulting page is vulnerable
            if is_vulnerable(res):
                sqli = True
                print("[+] SQL Injection vulnerability detected, link:", url)
                print("[+] Form:")
                pprint(form_details)
                break
            



# url = "http://testphp.vulnweb.com/artists.php?artist=1"
# url = "https://knaw.nl/"


def check_sqli(url):
    return scan_sql_injection(url)

# check_sqli(url)
# check_xss(url)


In [14]:
url = "https://ghccfcgff.com/"

check_sqli(url)


[!] Trying https://ghccfcgff.com/"


ConnectionError: HTTPSConnectionPool(host='ghccfcgff.com', port=443): Max retries exceeded with url: /%22 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7facbed709d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [18]:
import validators
 


URL is invalid


In [23]:
validation = validators.url("https://cxxzerzszsdßsß∂.nl")

if validation:
    print("URL is valid")
else:
    print("URL is invalid")

URL is valid


In [1]:
import pandas as pd

In [13]:
details

{'action': '/zoeken',
 'method': 'get',
 'inputs': [{'type': 'text', 'name': 'query'}]}

In [9]:
print(get_form_details(form))



NameError: name 'form' is not defined

In [ ]:
  <ul>
                {%for i in range(0, len)%} 
          
                <li>{{result[i]}}</li> 
            {%endfor%} 
                  
                </ul>
            <br>
            {{ jsonfile }}
            <br>